# Homework 4 - Alberto Trashaj
Download the Nile data, available in the R dataset: and fit a LLM to the data. Compare your results with Table 2.1.

### Import libraries
As usual we import the libraries needed to deal with algebra calculation and to plot the result.

In [1]:
import numpy as np
from scipy.optimize import minimize 
import matplotlib.pyplot as plt
import pandas as pd


### Functions
Now, as we did in the Lab, we will define the function of the Kalman filter, the likelihood and the estimator function. 

## Kalman filter

In [3]:
np.random.seed(0)

def KF(y, q):
    m20 = y[0]
    P20 = 1 + q

    n = len(y)
    mu_pred = np.empty(n)
    P = np.zeros(n)
    v = np.empty(n)
    K = np.zeros(n)
    F = np.zeros(n)
    dllk = np.empty(n)
    llk = 0

    mu_pred[1] = m20
    P[1] = P20


    for t in range(1, n-1):
        v[t] = y[t] - mu_pred[t]
        F[t] = P[t] + 1
        K[t] = P[t] / F[t]
        P[t + 1] = P[t] * (1 - K[t]) + q
        mu_pred[t + 1] = mu_pred[t] + K[t] * v[t]
        dllk[t] = -0.5 * np.log(F[t])

        print("Iteration:", t)
        print("Likelihood:", -0.5 * np.log(F[t]))
        print("Score:", -K[t])
        print("Signal-to-Noise Ratio (q):", q)
        print()

    #Since we need to define sigma_e_hat and the likelihood 
    # we need the last element computed in the for loop, so we retrieve those values below

    F[n - 1] = P[n - 1] + 1
    K[n - 1] = P[n - 1] / F[n - 1]
    dllk[n - 1] = -0.5 * np.log(F[n - 1])
    v[n - 1] = y[n - 1] - mu_pred[n - 1]

    sigma_e_hat = np.sum(v[1:] ** 2 / F[1:]) / (n - 1)
    llk = -0.5 * (n - 1) * np.log(sigma_e_hat) + np.sum(dllk[1:])

    return {'mu_pred': mu_pred, 'llk': llk, 'v': v, 'F': F}


## Loglikelihood 

In [14]:
def loglikelihood(par, y):
    q_new = par[0]
    obj = KF(y, q_new)['llk']
    return -obj

## Estimator 

Here a mention has to be done with the function minimize: in the book, they used the 'BFGS' algorithm, although to get the most similar result to the table I used the 'Nelder-Mead' method just to be consistent with the table 2.1. 
If in the method argument we put 'BFGS' the algorithm converges but not with the same result of the table.

In [4]:
#This uses the Nelder-Mead method
def estimator(y, par):
    n = len(y)
    q_0 = par

    hat = minimize(loglikelihood, q_0, args=(y,), method='Nelder-Mead') #here the method used is the Nelder-Mead although in the book they use BFGS 

    q_hat = hat.x[0]
    theta_list = {'q_hat': q_hat, 'iter': hat.nit}
    out = {'theta_list': theta_list}
    return out

In [20]:
##This uses the BFGS method
def estimator_bfgs(y, par):
    n = len(y)
    q_0 = par

    hat = minimize(loglikelihood, q_0, args=(y,), method='BFGS') #here the method used is the Nelder-Mead although in the book they use BFGS 

    q_hat = hat.x[0]
    theta_list = {'q_hat': q_hat, 'iter': hat.nit}
    out = {'theta_list': theta_list}
    return out


In [22]:
#Load the dataset

# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
# uncomment those lines above if you get an  URL error message


from statsmodels.datasets import get_rdataset
data = get_rdataset('Nile').data

#data = pd.read_csv("/Users/albertotrashaj/Desktop/Advanced-time-series-github/Assignement 4/Nile.csv")
y = np.array(data['value'])
n = len(y)

q_0 = 1
est = estimator(y, q_0)

#est = estimator_bfgs(y, q_0) uncomment this line and comment the above one to get the result with the BFGS algorithm

q_hat = est['theta_list']['q_hat']

filter_output = KF(y, q_hat)
sigma_e_hat = np.sum(filter_output['v'][1:] ** 2 / filter_output['F'][1:]) / (n - 1)
sigma_eta_hat = q_hat * sigma_e_hat


print("The value of sigma_e_hat is ", sigma_e_hat)
print("The value of sigma_eta_hat is ", sigma_eta_hat)

Iteration: 1
Likelihood: -0.5493061443340549
Score: -0.6666666666666666
Signal-to-Noise Ratio (q): 1.0

Iteration: 2
Likelihood: -0.49041462650586315
Score: -0.625
Signal-to-Noise Ratio (q): 1.0

Iteration: 3
Likelihood: -0.4825404480217935
Score: -0.6190476190476191
Signal-to-Noise Ratio (q): 1.0

Iteration: 4
Likelihood: -0.48140537375452397
Score: -0.6181818181818182
Signal-to-Noise Ratio (q): 1.0

Iteration: 5
Likelihood: -0.4812400571717648
Score: -0.6180555555555556
Signal-to-Noise Ratio (q): 1.0

Iteration: 6
Likelihood: -0.48121594393600503
Score: -0.6180371352785146
Signal-to-Noise Ratio (q): 1.0

Iteration: 7
Likelihood: -0.48121242599273545
Score: -0.6180344478216818
Signal-to-Noise Ratio (q): 1.0

Iteration: 8
Likelihood: -0.4812119127345054
Score: -0.6180340557275542
Signal-to-Noise Ratio (q): 1.0

Iteration: 9
Likelihood: -0.481211837851198
Score: -0.6180339985218034
Signal-to-Noise Ratio (q): 1.0

Iteration: 10
Likelihood: -0.48121182692587194
Score: -0.6180339901755971


/var/folders/4z/ffhfh8nj0zz1mqdx8xrwbjfr0000gn/T/ipykernel_34264/225816577.py:26: RuntimeWarning: invalid value encountered in log
  dllk[t] = -0.5 * np.log(F[t])
/var/folders/4z/ffhfh8nj0zz1mqdx8xrwbjfr0000gn/T/ipykernel_34264/225816577.py:29: RuntimeWarning: invalid value encountered in log
  print("Likelihood:", -0.5 * np.log(F[t]))
